In [ ]:
from notebook_environment import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
with gzip.open('../data/ccle.pickle.gz') as f:
    CCLE = pickle.load(f)

In [ ]:
kras_relevant_genes = pd.read_table(
    '../results/kras_relevant_genes.txt', squeeze=True)

rpkm__kras_relevant_gene_x_ccle_cellline = CCLE['Gene Expression']['df'].loc[
    kras_relevant_genes, :]

rpkm__kras_relevant_gene_x_ccle_cellline

In [ ]:
# Drop columns with only 1 unique object
df = ccal.drop_df_slices(
    rpkm__kras_relevant_gene_x_ccle_cellline, 0, max_n_unique_objects=1)

# Drop rows with only 1 unique object
df = ccal.drop_df_slices(df, 1, max_n_unique_objects=1)

# Rank normalize
array_2d = ccal.normalize_2d_array(df.values, 'rank', axis=0)

# Scale
array_2d *= 10000

# Convert to DataFrame
rpkm__kras_relevant_gene_x_ccle_cellline = pd.DataFrame(
    array_2d, index=df.index, columns=df.columns)

In [ ]:
# nmfs, nmfccs, cccs = ccal.define_components(
#     rpkm__kras_relevant_gene_x_ccle_cellline,
#     range(2, 11),
#     '../results/nmfccs',
#     algorithm='ls',
#     n_clusterings=30,
#     n_jobs=9,
#     random_seed=6137)

In [ ]:
w_matrix = pd.read_table('../results/nmfccs/nmf_k9_w.txt', index_col=0)
h_matrix = pd.read_table('../results/nmfccs/nmf_k9_h.txt', index_col=0)

In [ ]:
# Re-label components to have same names as in the paper
indices = [
    'C1',
    'C3',
    'C9',
    'C8',
    'C6',
    'C7',
    'C5',
    'C2',
    'C4',
]

w_matrix.columns = indices
h_matrix.index = indices

w_matrix.to_csv('../results/nmfccs/nmf_k9_w.txt', sep='\t')
h_matrix.to_csv('../results/nmfccs/nmf_k9_h.txt', sep='\t')

In [ ]:
ccal.plot_nmf(w_matrix, h_matrix)

In [ ]:
ccal.make_comparison_panel(
    h_matrix,
    h_matrix,
    axis=1,
    array_2d_0_name='Component',
    array_2d_1_name='Component')